In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# HR Diagram

In [2]:
def process(df):
    """
    Sort stars by O type and not O type, and return their luminosities and temperatures
    """
    
    o_l, o_t, r_t, r_l = [],[],[],[]

    for idx,row in df.iterrows():
    
        try:
        
            if 'O' in row['Type']:
            
                o_l.append(row['L/L_Sun'])
                o_t.append(row['T_e'])
            
            else:
            
                r_l.append(row['L/L_Sun'])
                r_t.append(row['T_e'])
                
        except: 
            pass
            
    return np.array(o_l), np.array(o_t),np.array(r_l),np.array(r_t)   

In [3]:
main = pd.read_fwf('main_theory.txt')
giant = pd.read_fwf('giant_theory.txt')
supergiant = pd.read_fwf('supergiant_theory.txt')

In [80]:
plt.gca().invert_xaxis()
plt.xscale('log')
plt.yscale('log')

plt.xlabel('$T_{eff}$')
plt.ylabel('$L/L_{sun}$')

o_l,o_t,r_l,r_t = process(main)
plt.scatter(o_t,o_l,facecolors='none',color='black')
plt.scatter(r_t,r_l,color='black')

o_l,o_t,r_l,r_t = process(giant)
plt.scatter(o_t,o_l,facecolors='none',color='red')
plt.scatter(r_t,r_l,color='red')

o_l,o_t,r_l,r_t = process(supergiant)
plt.scatter(o_t,o_l,facecolors='none',color='blue')
plt.scatter(r_t,r_l,color='blue')

plt.scatter(5777,1,marker='+',s=300,color='black')

s=5.67e-8
R=696.3e6
L=3.827e26
t_e = np.linspace(2500,55000,1000)
radius_index = np.array([.2,1,5,10,50,100,500,2500])
const_r = [4*np.pi*t_e**4*s*r*r*R*R/L for r in radius_index ]

for c in const_r:
    plt.plot(t_e,c,color='gray',linestyle='-')
plt.savefig('../tex/HR.png',dpi=350)
plt.show()    


# Molecular Weights

Import all the elemental mass fractions, and normalize them to add up to one instead of being relative to the hydrogen.

In [60]:
mass_fractions = pd.read_csv('abundances.csv')

for i in range(3,10):
    mass_fractions.ix[:,i] /= np.sum(mass_fractions.ix[:,i])

In [100]:
def mu_i(A,x):
    return 1./np.sum(x/A)

def mu_e(A,Z,x):
    return 1./np.sum(Z*x/A) # assume everything is fully ionized

def mu_t(A,Z,x):
    return 1./( (1./mu_i(A,x)) + (1./mu_e(A,Z,x)) )

def mathify(s):
    return "\\(%s\\)" % s

A = mass_fractions.values[:,2]
Z = mass_fractions.values[:,1]
out = {}
out['X'] = mass_fractions.values[0][3:]
out['Y'] = mass_fractions.values[1][3:]
out['Z'] = 1-out['X']-out['Y']

out[mathify('\\mu_I')] = [ mu_i(A,mass_fractions.values[:,i])  for i in range(3,10) ]
out[mathify('\\mu_e')] = [ mu_e(A,Z,mass_fractions.values[:,i])  for i in range(3,10) ]
out[mathify('\\mu')] = [ mu_t(A,Z,mass_fractions.values[:,i])  for i in range(3,10) ]

index = mass_fractions.columns.values.tolist()[3:]

out = pd.DataFrame(out,index=index)

In [106]:
print out.to_latex(escape=False) # actually works!

\begin{tabular}{llllrrr}
\toprule
{} &          X &           Y &             Z &   \(\mu\) &  \(\mu_e\) &  \(\mu_i\) \\
\midrule
Anders89   &  0.9096698 &  0.08887474 &   0.001455488 &  0.534063 &   1.055256 &   1.081314 \\
Asplund09  &  0.9206905 &  0.07835076 &  0.0009587071 &  0.530184 &   1.049246 &   1.071727 \\
Feldman92  &   0.909657 &  0.08887349 &   0.001469519 &  0.534068 &   1.055263 &   1.081327 \\
Anders82   &   0.924508 &  0.07405309 &    0.00143894 &  0.528887 &   1.047181 &   1.068582 \\
Grevesse98 &  0.9204288 &  0.07832849 &   0.001242676 &  0.530289 &   1.049388 &   1.072010 \\
Wilms00    &  0.9101783 &  0.08892442 &  0.0008972954 &  0.533854 &   1.054976 &   1.080751 \\
Lodders03  &  0.9257495 &  0.07331936 &  0.0009311188 &  0.528432 &   1.046511 &   1.067425 \\
\bottomrule
\end{tabular}



In [84]:
mass_fractions # I also discovered that pandas dataframes print as nice tables in ipython

,Element,Z,A,Anders89,Asplund09,Feldman92,Anders82,Grevesse98,Wilms00,Lodders03
0,H,1,1.007940,9.096698e-01,9.206905e-01,9.096570e-01,9.245080e-01,9.204288e-01,9.101783e-01,9.257495e-01
1,He,2,4.002602,8.887474e-02,7.835076e-02,8.887349e-02,7.405309e-02,7.832849e-02,8.892442e-02,7.331936e-02
2,Li,3,6.941000,1.319021e-11,1.031173e-11,1.146168e-11,2.024672e-09,1.159740e-11,0.000000e+00,1.758924e-09
3,Be,4,9.012182,1.282634e-11,2.209657e-11,2.283239e-11,2.653338e-11,2.310276e-11,0.000000e+00,2.379176e-11
4,B,5,10.811000,3.620486e-10,4.612660e-10,3.229282e-10,8.154160e-10,3.267522e-10,0.000000e+00,5.582270e-10
5,C,6,12.011000,3.302101e-04,2.476658e-04,3.620435e-04,4.114060e-04,3.046619e-04,2.184428e-04,2.268086e-04
6,N,7,14.006740,1.018830e-04,6.223868e-05,9.096570e-05,8.431513e-05,7.657968e-05,6.908253e-05,6.258067e-05
7,O,8,15.999400,7.741290e-04,4.511384e-04,7.741181e-04,6.832114e-04,6.222099e-04,4.459874e-04,4.536173e-04
8,F,9,18.998400,3.302101e-08,3.342107e-08,3.302055e-08,2.865975e-08,3.341157e-08,0.000000e+00,2.666159e-08
9,Ne,10,20.179700,1.118894e-04,7.835076e-05,1.173458e-04,1.275821e-04,1.104515e-04,7.927653e-05,6.859804e-05
